<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
</center>
<center>Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий

# <center> Capstone проект №1. Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 6.  Vowpal Wabbit

На этой неделе мы познакомимся с популярной библиотекой Vowpal Wabbit и попробуем ее на данных по посещению сайтов.

**План 6 недели:**
- Часть 1. Статья по Vowpal Wabbit
- Часть 2. Применение Vowpal Wabbit к данным по посещению сайтов
 - 2.1. Подготовка данных
 - 2.2. Валидация по отложенной выборке
 - 2.3. Валидация по тестовой выборке (Public Leaderboard)

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
Также будет полезна [презентация](https://github.com/esokolov/ml-course-msu/blob/master/ML15/lecture-notes/Sem08_vw.pdf) лектора специализации Евгения Соколова. И, конечно же, [документация](https://github.com/JohnLangford/vowpal_wabbit/wiki) Vowpal Wabbit.

<font color='red'>**Задание:**</font> заполните код в этой тетрадке и выберите ответы в [веб-форме](https://docs.google.com/forms/d/1wteunpEhAt_9s-WBwxYphB6XpniXsAZiFSNuFNmvOdk). 

## Часть 1. Статья про Vowpal Wabbit
Прочитайте [статью](https://habrahabr.ru/company/ods/blog/326418/) про Vowpal Wabbit на Хабре из серии открытого курса OpenDataScience по машинному обучению. Материал для этой статьи зародился из нашей специализации. Скачайте [тетрадку](https://github.com/Yorko/mlcourse_open/blob/master/jupyter_notebooks/topic08_sgd_hashing_vowpal_wabbit/topic8_sgd_hashing_vowpal_wabbit.ipynb), прилагаемую к статье, посмотрите код, изучите его, поменяйте, только так можно разобраться с Vowpal Wabbit.

## Часть 2. Применение Vowpal Wabbit к данным по посещению сайтов

### 2.1. Подготовка данных

**Далее посмотрим на Vowpal Wabbit в деле. Правда, в задаче нашего соревнования при бинарной классификации веб-сессий мы разницы не заметим – как по качеству, так и по скорости работы (хотя можете проверить), продемонстрируем всю резвость VW в задаче классификации на 400 классов. Исходные данные все те же самые, но выделено 400 пользователей, и решается задача их идентификации. Скачайте данные [отсюда](https://inclass.kaggle.com/c/identify-me-if-you-can4/data) – файлы `train_sessions_400users.csv` и `test_sessions_400users.csv`.**

In [1]:
import os
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression, SGDClassifier

In [2]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = 'D:'

**Загрузим обучающую и тестовую выборки. Можете заметить, что тестовые сессии здесь по времени четко отделены от сессий в обучающей выборке. **

In [3]:
train_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'train_sessions_400users.csv'), 
                           index_col='session_id')

In [4]:
test_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'test_sessions_400users.csv'), 
                           index_col='session_id')

In [10]:
test_df_400.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,9,2014-10-04 12:24:43,304.0,2014-10-04 12:25:34,308.0,2014-10-04 12:28:33,307.0,2014-10-04 12:28:33,91.0,2014-10-04 12:28:33,308.0,2014-10-04 12:28:34,312.0,2014-10-04 12:30:31,300.0,2014-10-04 12:31:30,305.0,2014-10-04 12:32:31,309.0,2014-10-04 12:34:31
2,838,2014-12-02 09:20:37,504.0,2014-12-02 09:20:38,68.0,2014-12-02 09:20:38,11.0,2014-12-02 09:20:38,838.0,2014-12-02 09:20:38,11.0,2014-12-02 09:20:40,838.0,2014-12-02 09:20:41,886.0,2014-12-02 09:20:42,27.0,2014-12-02 09:20:42,305.0,2014-12-02 09:20:42
3,190,2014-10-01 09:27:38,192.0,2014-10-01 09:27:38,8.0,2014-10-01 09:27:38,189.0,2014-10-01 09:27:38,191.0,2014-10-01 09:27:39,189.0,2014-10-01 09:27:39,190.0,2014-10-01 09:27:39,2375.0,2014-10-01 09:27:39,192.0,2014-10-01 09:27:39,8.0,2014-10-01 09:27:39
4,295,2014-10-02 14:34:17,295.0,2014-10-02 14:34:18,295.0,2014-10-02 14:34:19,295.0,2014-10-02 14:34:20,295.0,2014-10-02 14:34:21,295.0,2014-10-02 14:34:22,295.0,2014-10-02 14:34:23,295.0,2014-10-02 14:34:24,295.0,2014-10-02 14:34:25,295.0,2014-10-02 14:34:26
5,31,2014-05-19 17:50:21,3177.0,2014-05-19 17:50:21,3174.0,2014-05-19 17:50:21,32434.0,2014-05-19 17:50:22,27.0,2014-05-19 17:50:22,31.0,2014-05-19 17:50:22,5698.0,2014-05-19 17:50:25,5698.0,2014-05-19 17:50:26,5698.0,2014-05-19 17:50:27,4173.0,2014-05-19 17:50:27


**Видим, что в обучающей выборке 182793 сессий, в тестовой – 46473, и сессии действительно принадлежат 400 различным пользователям.**

In [6]:
train_df_400.shape, test_df_400.shape, train_df_400['user_id'].nunique()

((182793, 21), (46473, 20), 400)

**Vowpal Wabbit любит, чтоб метки классов были распределены от 1 до K, где K – число классов в задаче классификации (в нашем случае – 400). Поэтому придется применить `LabelEncoder`, да еще и +1 потом добавить (`LabelEncoder` переводит метки в диапозон от 0 до K-1). Потом надо будет применить обратное преобразование.**

In [49]:
from sklearn import preprocessing
import numpy
le = preprocessing.LabelEncoder()
y = numpy.array(le.fit_transform(train_df_400.user_id))
#class_encoder = numpy.array(le.classes_) 
y_for_vw = y + 1

In [51]:
y_for_vw.max(), y_for_vw.min()

(400, 1)

**Далее будем сравнивать VW с SGDClassifier и с логистической регрессией. Всем моделям этим нужна предобработка входных данных. Подготовьте для sklearn-моделей разреженные матрицы, как мы это делали в 5 части:**
- объедините обучающиую и тестовую выборки
- выберите только сайты (признаки от 'site1' до 'site10')
- замените пропуски на нули (сайты у нас нумеровались с 0)
- переведите в разреженный формат `csr_matrix`
- разбейте обратно на обучающую и тестовую части

In [9]:
sites = ['site' + str(i) for i in range(1, 11)]

In [22]:
trainandtest = pd.concat([train_df_400, test_df_400], ignore_index=True)
trainandtest = trainandtest[sites]
trainandtest.fillna(0, inplace = True)

In [26]:
def SparseMatrixMakerAlt(sitedat):
    #sitedat = dat.drop(['user_id', 'session_id'], axis=1)
    indices = []
    datas = []
    indptrs = [0]
    mylen = len(sitedat.columns)
    for j in range(len(sitedat)):
        #if j % 1000 == 0:
            #print(j)
        indices.extend((sitedat.iloc[j].values))
        datas.extend((numpy.repeat(1, mylen)))
        indptrs.append(len(indices))
    return(csr_matrix((datas, indices, indptrs), dtype = int))

In [27]:
rrr = SparseMatrixMakerAlt(trainandtest)

In [28]:
rrr2 = rrr[:,1:]

In [52]:
''' ВАШ КОД ЗДЕСЬ '''
X_train_sparse = rrr2[:len(train_df_400),:]
X_test_sparse = rrr2[len(train_df_400):,:]
y = y_for_vw

In [53]:
len(y),(X_train_sparse.shape)

(182793, (182793, 36656))

### 2.2. Валидация по отложенной выборке

**Выделим обучающую (70%) и отложенную (30%) части исходной обучающей выборки. Данные не перемешиваем, учитываем, что сессии отсортированы по времени.**

In [54]:
train_share = int(.7 * train_df_400.shape[0])
train_df_part = train_df_400[sites].iloc[:train_share, :]
valid_df = train_df_400[sites].iloc[train_share:, :]
X_train_part_sparse = X_train_sparse[:train_share, :]
X_valid_sparse = X_train_sparse[train_share:, :]

In [55]:
y_train_part = y[:train_share]
y_valid = y[train_share:]
y_train_part_for_vw = y_for_vw[:train_share]
y_valid_for_vw = y_for_vw[train_share:]

In [59]:
train_df_part.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,23713,23720.0,23713.0,23713.0,23720.0,23713.0,23713.0,23713.0,23713.0,23713.0
2,8726,8725.0,665.0,8727.0,45.0,8725.0,45.0,5320.0,5320.0,5320.0
3,303,19.0,303.0,303.0,303.0,303.0,303.0,309.0,303.0,303.0
4,1359,925.0,1240.0,1360.0,1344.0,1359.0,1346.0,1345.0,1344.0,1345.0
5,11,85.0,52.0,85.0,11.0,52.0,11.0,85.0,10.0,85.0


**Реализуйте функцию, `arrays_to_vw`, переводящую обучающую выборку в формат Vowpal Wabbit.**

Вход:
 - X – матрица `NumPy` (обучающая выборка)
 - y (необяз.) - вектор ответов (`NumPy`). Необязателен, поскольку тестовую матрицу будем обрабатывать этой же функцией
 - train – флаг, True в случае обучающей выборки, False – в случае тестовой выборки
 - out_file – путь к файлу .vw, в который будет произведена запись
 
Детали:
- надо пройтись по всем строкам матрицы `X` и записать через пробел все значения, предварительно добавив вперед нужную метку класса из вектора `y` и знак-разделитель `|`
- в тестовой выборке на месте меток целевого класса можно писать произвольные, допустим, 1

In [70]:
def arrays_to_vw(X, y=None, train=True, out_file='tmp.vw'):
    X.reset_index(drop = True, inplace = True)
    for j in range(len(X)):
        if train:
            prob = y[j]
        else:
            prob = 1
        astr = str(prob) + ' | '
        for acol in X.columns:
            astr += str(X.loc[j, acol]) + ' '
        astr += '\n'
        with open(out_file, 'a', encoding = 'utf-8') as g:
            g.write(astr)
        g.close()

**Примените написанную функцию к части обучащей выборки `(train_df_part, y_train_part_for_vw)`, к отложенной выборке `(valid_df, y_valid_for_vw)`, ко всей обучающей выборке и ко всей тестовой выборке. Обратите внимание, что на вход наш метод принимает именно матрицы и вектора `NumPy`.**

In [72]:
%%time
# будет 4 вызова
arrays_to_vw(train_df_part, y_train_part_for_vw, True, 'D:/trainpart.vw')
arrays_to_vw(valid_df, y_valid_for_vw, True, 'D:/validpart.vw')
arrays_to_vw(train_df_400[sites], y, True, 'D:/train.vw')
arrays_to_vw(test_df_400[sites], None, False, 'D:/test.vw')



Wall time: 24min 42s


In [90]:
arrays_to_vw(train_df_400[sites], y, True, 'D:/train.vw')
arrays_to_vw(test_df_400[sites], None, False, 'D:/test.vw')


**Результат должен получиться таким.**

In [7]:
!head -3 $PATH_TO_DATA/train_part.vw

262 | 23713 23720 23713 23713 23720 23713 23713 23713 23713 23713
82 | 8726 8725 665 8727 45 8725 45 5320 5320 5320
16 | 303 19 303 303 303 303 303 309 303 303


In [8]:
!head -3  $PATH_TO_DATA/valid.vw

4 | 7 923 923 923 11 924 7 924 838 7
160 | 91 198 11 11 302 91 668 311 310 91
312 | 27085 848 118 118 118 118 11 118 118 118


In [9]:
!head -3 $PATH_TO_DATA/test.vw

1 | 9 304 308 307 91 308 312 300 305 309
1 | 838 504 68 11 838 11 838 886 27 305
1 | 190 192 8 189 191 189 190 2375 192 8


**Обучите модель Vowpal Wabbitна выборке `train_part.vw`. Укажите, что решается задача классификации с 400 классами (`--oaa`), сделайте 3 прохода по выборке (`--passes`). Задайте некоторый кэш-файл (`--cache_file`, можно просто указать флаг `-c`), так VW будет быстрее делать все следующие после первого проходы по выборке (прошлый кэш-файл удаляется с помощью аргумента `-k`). Также укажите значение параметра `b`=26. Это число бит, используемых для хэширования, в данном случае нужно больше, чем 18 по умолчанию. Наконец, укажите `random_seed`=17. Остальные параметры пока не меняйте, далее уже в свободном режиме соревнования можете попробовать другие функции потерь.**

In [ ]:
train_part_vw = os.path.join(PATH_TO_DATA, 'train_part.vw')
valid_vw = os.path.join(PATH_TO_DATA, 'valid.vw')
train_vw = os.path.join(PATH_TO_DATA, 'train.vw')
test_vw = os.path.join(PATH_TO_DATA, 'test.vw')
model = os.path.join(PATH_TO_DATA, 'vw_model.vw')
pred = os.path.join(PATH_TO_DATA, 'vw_pred.csv')

In [ ]:
%%time
!vw ''' ВАШ КОД ЗДЕСЬ '''

**Запишите прогнозы на выборке *valid.vw* в *vw_valid_pred.csv*.**

In [ ]:
%%time
!vw ''' ВАШ КОД ЗДЕСЬ '''

In [80]:
y_valid.min(), y_valid_for_vw.min()

(1, 1)

In [8]:
class_encoder[0:10], y_for_vw[0:10]

(array([ 1,  5,  6,  7,  8,  9, 10, 13, 16, 19], dtype=int64),
 array([ 2,  6,  7,  8,  9, 10, 11, 14, 17, 20], dtype=int64))

**Считайте прогнозы *kaggle_data/vw_valid_pred.csv*  из файла и посмотрите на долю правильных ответов на отложенной части.**

In [79]:
import sklearn.metrics
validdat = pd.read_csv('D:/validpartmodel.txt', header = None)
print(validdat.head())
print(sklearn.metrics.accuracy_score(validdat, y_valid_for_vw))

     0
0  188
1  220
2  332
3  143
4  345
0.329005434188


In [89]:
validdat = pd.read_csv('D:/validpartmodel2.txt', header = None)
print(validdat.head())
print(sklearn.metrics.accuracy_score(validdat, y_valid_for_vw))

     0
0  188
1  385
2  332
3  143
4  345
0.338852620446


**Теперь обучите `SGDClassifier` (3 прохода по выборке, логистическая функция потерь) и `LogisticRegression` на 70% разреженной обучающей выборки – `(X_train_part_sparse, y_train_part)`, сделайте прогноз для отложенной выборки `(X_valid_sparse, y_valid)` и посчитайте доли верных ответов. Логистическая регрессия будет обучаться долго (у меня – 15 минут) – это нормально. Укажите везде `random_state`=17, `n_jobs`=-1.**

In [84]:
logit = sklearn.linear_model.LogisticRegression(random_state=17, n_jobs=-1)
sgd_logit = sklearn.linear_model.SGDClassifier(loss = 'log', random_state=17, n_jobs=-1, max_iter=3) 

In [85]:
%%time
logit.fit(X_train_part_sparse, y_train_part)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Wall time: 19min 20s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [86]:
%%time
sgd_logit.fit(X_train_part_sparse, y_train_part)

Wall time: 15.5 s


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=3, n_iter=None,
       n_jobs=-1, penalty='l2', power_t=0.5, random_state=17, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [88]:
print(sklearn.metrics.accuracy_score(sgd_logit.predict(X_valid_sparse), y_valid_for_vw))
print(sklearn.metrics.accuracy_score(logit.predict(X_valid_sparse), y_valid_for_vw))


0.291075531566
0.362923520187


**<font color='red'>Вопрос 1. </font> Посчитайте долю правильных ответов на отложенной выборке для Vowpal Wabbit, округлите до 3 знаков после запятой.**

**<font color='red'>Вопрос 2. </font> Посчитайте долю правильных ответов на отложенной выборке для SGD, округлите до 3 знаков после запятой.**

**<font color='red'>Вопрос 3. </font> Посчитайте долю правильных ответов на отложенной выборке для логистической регрессии, округлите до 3 знаков после запятой.**

In [ ]:
vw_valid_acc = ''' ВАШ КОД ЗДЕСЬ '''
sgd_valid_acc = ''' ВАШ КОД ЗДЕСЬ '''
logit_valid_acc = ''' ВАШ КОД ЗДЕСЬ '''

In [ ]:
def write_answer_to_file(answer, file_address):
    with open(file_address, 'w') as out_f:
        out_f.write(str(answer))

In [ ]:
write_answer_to_file("{}  {} {}".format(round(vw_valid_acc, 3), round(sgd_valid_acc, 3),
                                     round(logit_valid_acc, 3)),  
                     'answer6_1.txt')

### 2.3. Валидация по тестовой выборке (Public Leaderboard)

**Обучите модель VW с теми же параметрами на всей обучающей выборке – *train.vw*.**

In [ ]:
%%time
!vw ''' ВАШ КОД ЗДЕСЬ '''

**Сделайте прогноз для тестовой выборки.**

In [91]:
testmodel1 = pd.read_csv('D:/testmodel1.txt', header = None)
testmodel2 = pd.read_csv('D:/testmodel2.txt', header = None)

In [98]:
testmodel1.head()

,0
0,207
1,21
2,311
3,265
4,372


**Запишите прогноз в файл, примените обратное преобразование меток (был LabelEncoder и потом +1 в меткам) и отправьте решение на Kaggle.**

In [123]:
import numpy
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_labels = numpy.array(predicted_labels.values)-1
    predicted_labels=le.inverse_transform(predicted_labels)
    predicted_df = pd.DataFrame(predicted_labels,
                                index = numpy.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)
    return(predicted_df)

In [119]:
vw_pred = numpy.array(testmodel1.values)-1

In [120]:
vw_pred = le.inverse_transform(vw_pred)

In [122]:
vw_pred.max()

997

In [124]:
write_to_submission_file(testmodel1, 'D:/vw_model_1cor.txt')

,user_id
1,527
2,48
3,795
4,656
5,946
6,361
7,946
8,352
9,917
10,653


In [125]:
write_to_submission_file(testmodel2, 'D:/vw_model2_cor.txt')

,user_id
1,527
2,149
3,946
4,656
5,946
6,361
7,946
8,352
9,917
10,468


In [112]:
testmodel2.min(), testmodel2.max()

(0    1
 dtype: int64, 0    400
 dtype: int64)

In [114]:
prdf = write_to_submission_file(testmodel2, 'D:/vw_model__2.txt')

In [115]:
prdf.min(), prdf.max()

(user_id    0
 dtype: int64, user_id    399
 dtype: int64)

In [116]:
prdf.head()

,user_id
1,206
2,58
3,371
4,264
5,371


**Сделайте то же самое для SGD и логистической регрессии. Тут уже ждать обучение логистической регрессии совсем скучно (заново запускать тетрадку вам не захочется), но давайте дождемся.**

In [106]:
%%time
logit.fit(X_train_sparse, y_for_vw)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Wall time: 32min 33s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [108]:
%%time
sgd_logit.fit(X_train_sparse, y_for_vw)

Wall time: 16.9 s


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=3, n_iter=None,
       n_jobs=-1, penalty='l2', power_t=0.5, random_state=17, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [126]:
write_to_submission_file(pd.DataFrame(logit.predict(X_test_sparse)), 'D:/logit_model_cor.txt')
write_to_submission_file(pd.DataFrame(sgd_logit.predict(X_test_sparse)), 'D:/sgd_logit_model_cor.txt')


,user_id
1,984
2,149
3,580
4,656
5,946
6,798
7,946
8,803
9,575
10,405


In [117]:
mylogit = write_to_submission_file(pd.DataFrame(logit.predict(X_test_sparse)), 'D:/logit_model.txt')


In [118]:
mylogit.max(), mylogit.min()

(user_id    399
 dtype: int64, user_id    0
 dtype: int64)

In [ ]:
#0.19306, 0.19887, 0.17490

In [ ]:
write_to_submission_file(sgd_logit_test_pred, 
                         os.path.join(PATH_TO_DATA, 'logit_400_users.csv'))
write_to_submission_file(sgd_logit_test_pred, 
                         os.path.join(PATH_TO_DATA, 'sgd_400_users.csv'))

Посмотрим на доли правильных ответов на публичной части (public leaderboard) тестовой выборки [этого](https://inclass.kaggle.com/c/identify-me-if-you-can4) соревнования.

**<font color='red'>Вопрос 4. </font> Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard)  для Vowpal Wabbit?**

**<font color='red'>Вопрос 5. </font> Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard)  для SGD?**

**<font color='red'>Вопрос 6. </font> Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard)  для логистической регрессии?**


**В заключение по заданию:**
- Про соотношение качества классификации и скорости обучения VW, SGD и logit выводы предлагается сделать самостоятельно
- Пожалуй, задача классификации на 400 классов (идентификация 400 пользователей) решается недостаточно хорошо при честном отделении по времени тестовой выборки от обучающей. Далее мы будем соревноваться в идентификации одного пользователя (Элис) – [вот](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнование, в котором предлагается поучаствовать. Не перепутайте! 

**Удачи!**